# practice

### torch

In [1]:
import sys
sys.path.insert(0, '/media/allen/mass/deep-learning-works/')
import os.path as osp
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from database.dataset_factory import get_dataset
from manager.manager_factory import get_manager
from model.model_factory import get_model

from config.config_factory import _C as cfg
from config.config_factory import build_output

from tools.logger import setup_logger
logger = setup_logger(".")
from tools.utils import deploy_macro

import cv2
from PIL import Image 

def to_pil(cv_img):
    img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    return Image.fromarray(img)

from tools.utils import multi_pose_decode, multi_pose_post_process, _sigmoid

ModuleNotFoundError: No module named 'database'

In [2]:
config_path = '/media/allen/mass/deep-learning-works/config/keypoint.yml'
cfg.merge_from_file(config_path)
cfg.MODEL.NAME = 'osnet_pfpn'
cfg.DB.PATH     = '/media/allen/mass/DB/'
cfg.DB.USE_TRAIN = False
cfg.DB.USE_TEST  = True
cfg.MODEL.GPU   = [1]
cfg.MODEL.NORM  = "BN"
cfg.COCO.TARGET = 'original'
cfg.ENGINE      = 'center_kp'
cfg.MANAGER     = 'center_kp'
cfg.DB.DATA     = 'cifar10'
cfg.DB.DATASET  = 'cifar10'
cfg.DB.LOADER   = 'cifar10'
cfg.DB.NUM_CLASSES   = 1
cfg.DB.NUM_KEYPOINTS = 17
cfg.EVALUATE = ''


In [3]:
manager = get_manager(cfg.MANAGER)(cfg)

In [4]:
manager.check_size((1,3,512,512))

2020-01-02 10:26:27,604 logger INFO: backbone.conv.0.conv.0.0                                    
2020-01-02 10:26:27,605 logger INFO:              input,   1 x   3 x 512 x 512
2020-01-02 10:26:27,606 logger INFO:             output,   1 x  64 x 256 x 256
2020-01-02 10:26:27,628 logger INFO: backbone.conv.0.conv.0.1                                    
2020-01-02 10:26:27,629 logger INFO:              input,   1 x  64 x 256 x 256
2020-01-02 10:26:27,629 logger INFO:             output,   1 x  64 x 256 x 256
2020-01-02 10:26:27,635 logger INFO: backbone.conv.0.conv.0.2                                    
2020-01-02 10:26:27,636 logger INFO:              input,   1 x  64 x 256 x 256
2020-01-02 10:26:27,637 logger INFO:             output,   1 x  64 x 256 x 256
2020-01-02 10:26:27,638 logger INFO: backbone.conv.0.conv.0                                      
2020-01-02 10:26:27,639 logger INFO:              input,   1 x   3 x 512 x 512
2020-01-02 10:26:27,640 logger INFO:             output

2020-01-02 10:26:27,842 logger INFO:              input,   1 x  64 x 128 x 128
2020-01-02 10:26:27,844 logger INFO:             output,   1 x  64 x 128 x 128
2020-01-02 10:26:27,845 logger INFO: backbone.stage1.0.conv2c.0.conv                             
2020-01-02 10:26:27,846 logger INFO:              input,   1 x  64 x 128 x 128
2020-01-02 10:26:27,847 logger INFO:             output,   1 x  64 x 128 x 128
2020-01-02 10:26:27,850 logger INFO: backbone.stage1.0.conv2c.0                                  
2020-01-02 10:26:27,851 logger INFO:              input,   1 x  64 x 128 x 128
2020-01-02 10:26:27,852 logger INFO:             output,   1 x  64 x 128 x 128
2020-01-02 10:26:27,860 logger INFO: backbone.stage1.0.conv2c.1.conv.0                           
2020-01-02 10:26:27,861 logger INFO:              input,   1 x  64 x 128 x 128
2020-01-02 10:26:27,863 logger INFO:             output,   1 x  64 x 128 x 128
2020-01-02 10:26:27,869 logger INFO: backbone.stage1.0.conv2c.1.conv.1    

2020-01-02 10:26:28,011 logger INFO:             output,   1 x  64 x 128 x 128
2020-01-02 10:26:28,012 logger INFO: backbone.stage1.0.conv2d.3                                  
2020-01-02 10:26:28,013 logger INFO:              input,   1 x  64 x 128 x 128
2020-01-02 10:26:28,014 logger INFO:             output,   1 x  64 x 128 x 128
2020-01-02 10:26:28,014 logger INFO: backbone.stage1.0.conv2d                                    
2020-01-02 10:26:28,015 logger INFO:              input,   1 x  64 x 128 x 128
2020-01-02 10:26:28,016 logger INFO:             output,   1 x  64 x 128 x 128
2020-01-02 10:26:28,017 logger INFO: backbone.stage1.0.gate.gate.0                               
2020-01-02 10:26:28,018 logger INFO:              input,   1 x  64 x 128 x 128
2020-01-02 10:26:28,019 logger INFO:             output,   1 x  64 x   1 x   1
2020-01-02 10:26:28,020 logger INFO: backbone.stage1.0.gate.gate.1                               
2020-01-02 10:26:28,021 logger INFO:              input

2020-01-02 10:26:28,156 logger INFO: backbone.stage1.0.conv3.conv.0                              
2020-01-02 10:26:28,157 logger INFO:              input,   1 x  64 x 128 x 128
2020-01-02 10:26:28,157 logger INFO:             output,   1 x 256 x 128 x 128
2020-01-02 10:26:28,159 logger INFO: backbone.stage1.0.conv3.conv                                
2020-01-02 10:26:28,160 logger INFO:              input,   1 x  64 x 128 x 128
2020-01-02 10:26:28,161 logger INFO:             output,   1 x 256 x 128 x 128
2020-01-02 10:26:28,161 logger INFO: backbone.stage1.0.conv3                                     
2020-01-02 10:26:28,162 logger INFO:              input,   1 x  64 x 128 x 128
2020-01-02 10:26:28,163 logger INFO:             output,   1 x 256 x 128 x 128
2020-01-02 10:26:28,176 logger INFO: backbone.stage1.0.downsample.conv.0.0                       
2020-01-02 10:26:28,177 logger INFO:              input,   1 x  64 x 128 x 128
2020-01-02 10:26:28,177 logger INFO:             output

2020-01-02 10:26:28,323 logger INFO:              input,   1 x  64 x 128 x 128
2020-01-02 10:26:28,324 logger INFO:             output,   1 x  64 x 128 x 128
2020-01-02 10:26:28,328 logger INFO: backbone.stage1.1.conv2c.0.conv.0                           
2020-01-02 10:26:28,329 logger INFO:              input,   1 x  64 x 128 x 128
2020-01-02 10:26:28,330 logger INFO:             output,   1 x  64 x 128 x 128
2020-01-02 10:26:28,335 logger INFO: backbone.stage1.1.conv2c.0.conv.1                           
2020-01-02 10:26:28,335 logger INFO:              input,   1 x  64 x 128 x 128
2020-01-02 10:26:28,337 logger INFO:             output,   1 x  64 x 128 x 128
2020-01-02 10:26:28,340 logger INFO: backbone.stage1.1.conv2c.0.conv.2                           
2020-01-02 10:26:28,342 logger INFO:              input,   1 x  64 x 128 x 128
2020-01-02 10:26:28,342 logger INFO:             output,   1 x  64 x 128 x 128
2020-01-02 10:26:28,343 logger INFO: backbone.stage1.1.conv2c.0.conv      

...

2020-01-02 10:26:30,091 logger INFO:             output,   1 x 128 x  32 x  32
2020-01-02 10:26:30,092 logger INFO: backbone.stage3.1.conv2c.2.conv.2                           
2020-01-02 10:26:30,093 logger INFO:              input,   1 x 128 x  32 x  32
2020-01-02 10:26:30,094 logger INFO:             output,   1 x 128 x  32 x  32
2020-01-02 10:26:30,095 logger INFO: backbone.stage3.1.conv2c.2.conv                             
2020-01-02 10:26:30,096 logger INFO:              input,   1 x 128 x  32 x  32
2020-01-02 10:26:30,096 logger INFO:             output,   1 x 128 x  32 x  32
2020-01-02 10:26:30,097 logger INFO: backbone.stage3.1.conv2c.2                                  
2020-01-02 10:26:30,097 logger INFO:              input,   1 x 128 x  32 x  32
2020-01-02 10:26:30,098 logger INFO:             output,   1 x 128 x  32 x  32
2020-01-02 10:26:30,099 logger INFO: backbone.stage3.1.conv2c                                    
2020-01-02 10:26:30,099 logger INFO:              input

2020-01-02 10:26:30,178 logger INFO: backbone.stage3.1.gate.gate.0                               
2020-01-02 10:26:30,178 logger INFO:              input,   1 x 128 x  32 x  32
2020-01-02 10:26:30,179 logger INFO:             output,   1 x 128 x   1 x   1
2020-01-02 10:26:30,180 logger INFO: backbone.stage3.1.gate.gate.1                               
2020-01-02 10:26:30,181 logger INFO:              input,   1 x 128 x   1 x   1
2020-01-02 10:26:30,181 logger INFO:             output,   1 x   8 x   1 x   1
2020-01-02 10:26:30,182 logger INFO: backbone.stage3.1.gate.gate.2                               
2020-01-02 10:26:30,182 logger INFO:              input,   1 x   8 x   1 x   1
2020-01-02 10:26:30,183 logger INFO:             output,   1 x   8 x   1 x   1
2020-01-02 10:26:30,188 logger INFO: backbone.stage3.1.gate.gate.3                               
2020-01-02 10:26:30,188 logger INFO:              input,   1 x   8 x   1 x   1
2020-01-02 10:26:30,189 logger INFO:             output

2020-01-02 10:26:30,295 logger INFO:              input,   1 x 256 x  32 x  32
2020-01-02 10:26:30,296 logger INFO:             output,   1 x 256 x  32 x  32
2020-01-02 10:26:30,296 logger INFO: backbone.fpn3.conv.0                                        
2020-01-02 10:26:30,298 logger INFO:              input,   1 x 512 x  32 x  32
2020-01-02 10:26:30,299 logger INFO:             output,   1 x 256 x  32 x  32
2020-01-02 10:26:30,299 logger INFO: backbone.fpn3.conv                                          
2020-01-02 10:26:30,300 logger INFO:              input,   1 x 512 x  32 x  32
2020-01-02 10:26:30,301 logger INFO:             output,   1 x 256 x  32 x  32
2020-01-02 10:26:30,301 logger INFO: backbone.fpn3                                               
2020-01-02 10:26:30,302 logger INFO:              input,   1 x 512 x  32 x  32
2020-01-02 10:26:30,303 logger INFO:             output,   1 x 256 x  32 x  32
2020-01-02 10:26:30,306 logger INFO: backbone.fpn2.conv.0.0               

2020-01-02 10:26:30,458 logger INFO:             output,   1 x 128 x 128 x 128
2020-01-02 10:26:30,458 logger INFO: backbone.seg3_2_upsample                                    
2020-01-02 10:26:30,459 logger INFO:              input,   1 x 128 x  64 x  64
2020-01-02 10:26:30,459 logger INFO:             output,   1 x 128 x 128 x 128
2020-01-02 10:26:30,464 logger INFO: backbone.seg2_1.conv.0.0                                    
2020-01-02 10:26:30,465 logger INFO:              input,   1 x 256 x  32 x  32
2020-01-02 10:26:30,466 logger INFO:             output,   1 x 128 x  32 x  32
2020-01-02 10:26:30,467 logger INFO: backbone.seg2_1.conv.0.1                                    
2020-01-02 10:26:30,468 logger INFO:              input,   1 x 128 x  32 x  32
2020-01-02 10:26:30,468 logger INFO:             output,   1 x 128 x  32 x  32
2020-01-02 10:26:30,470 logger INFO: backbone.seg2_1.conv.0.2                                    
2020-01-02 10:26:30,471 logger INFO:              input

2020-01-02 10:26:30,663 logger INFO: backbone.offset_reg.regressor.2                             
2020-01-02 10:26:30,664 logger INFO:              input,   1 x   2 x 128 x 128
2020-01-02 10:26:30,665 logger INFO:             output,   1 x   2 x 128 x 128
2020-01-02 10:26:30,666 logger INFO: backbone.offset_reg.regressor                               
2020-01-02 10:26:30,667 logger INFO:              input,   1 x 128 x 128 x 128
2020-01-02 10:26:30,668 logger INFO:             output,   1 x   2 x 128 x 128
2020-01-02 10:26:30,669 logger INFO: backbone.offset_reg                                         
2020-01-02 10:26:30,670 logger INFO:              input,   1 x 128 x 128 x 128
2020-01-02 10:26:30,671 logger INFO:             output,   1 x   2 x 128 x 128
2020-01-02 10:26:30,676 logger INFO: backbone.size_reg.regressor.0                               
2020-01-02 10:26:30,677 logger INFO:              input,   1 x 128 x 128 x 128
2020-01-02 10:26:30,680 logger INFO:             output

In [5]:
from database.data_factory import get_data
cifar10 = get_data('cifar10')(cfg)

Files already downloaded and verified
2019-12-30 09:09:35,379 logger INFO: => CIFAR10 TRAIN loaded
2019-12-30 09:09:35,380 logger INFO: Dataset statistics:
2019-12-30 09:09:35,381 logger INFO:   ------------------------------
2019-12-30 09:09:35,382 logger INFO:   subset   | # class | # images
2019-12-30 09:09:35,382 logger INFO:   ------------------------------
2019-12-30 09:09:35,383 logger INFO:   train    |      10 |    50000
2019-12-30 09:09:35,384 logger INFO:   ------------------------------
Files already downloaded and verified
2019-12-30 09:09:36,078 logger INFO: => CIFAR10 VAL loaded
2019-12-30 09:09:36,079 logger INFO: Dataset statistics:
2019-12-30 09:09:36,079 logger INFO:   ------------------------------
2019-12-30 09:09:36,080 logger INFO:   subset   | # class | # images
2019-12-30 09:09:36,081 logger INFO:   ------------------------------
2019-12-30 09:09:36,082 logger INFO:   val      |      10 |    10000
2019-12-30 09:09:36,083 logger INFO:   -------------------------

In [15]:
X = []
y = []
for i, (pil_img, label) in enumerate(cifar10.handle['train']):
    X.append(i)
    y.append(label)

In [25]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(X, y)

train_index, test_index = next(iter(sss.split(X, y)))

In [27]:
import os

In [28]:
os.makedirs('/media/allen/mass/gcp_cifar10/train')
os.makedirs('/media/allen/mass/gcp_cifar10/test')

In [43]:
dst = '/media/allen/mass/gcp_cifar10'

In [40]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [47]:
for i in range(cifar10.n_samples['train']):
    img = cifar10.handle['train'][i][0]
    fname = os.path.join(dst, '{:06}.jpg'.format(i))
    img.save(fname, "JPEG")

In [36]:
import pandas as pd

In [73]:
df = pd.DataFrame(columns=['split', 'file', 'label'])
split = []
fnames = []
labels = []
for i in range(cifar10.n_samples['train']):
    labels.append(cifar10.handle['train'][i][1])
    fname = 'gs://cifar10-263315-vcm/cifar10/{:06}.jpg'.format(i)
    if i in train_index:
        split.append('TRAIN')
    if i in test_index:
        split.append('VALIDATION')
    fnames.append(fname)
for i in range(cifar10.n_samples['val']):
    labels.append(cifar10.handle['val'][i][1])
    fname = 'gs://cifar10-263315-vcm/cifar10/{:06}.jpg'.format(i+50000)
    fnames.append(fname)
    split.append('TEST')

In [74]:
df['file'] = pd.Series(fnames)
df['label'] = pd.Series(labels)
df['split'] = pd.Series(split)

In [75]:
df.head()

,split,file,label
0,VALIDATION,gs://cifar10-263315-vcm/cifar10/000000.jpg,6
1,TRAIN,gs://cifar10-263315-vcm/cifar10/000001.jpg,9
2,TRAIN,gs://cifar10-263315-vcm/cifar10/000002.jpg,9
3,TRAIN,gs://cifar10-263315-vcm/cifar10/000003.jpg,4
4,VALIDATION,gs://cifar10-263315-vcm/cifar10/000004.jpg,1


In [76]:
df.to_csv('/media/allen/mass/gcp_cifar10.csv', header=False, index=False)

In [ ]:
im.save(file + ".thumbnail", "JPEG")

In [1]:
import sys
sys.path.insert(0, '/media/allen/mass/deep-learning-works/')
from config.config_factory import _C as cfg
cfg.merge_from_file('../config/cifar10.yml')

In [3]:
a = []
def change(a):
    a.extend([1])

In [4]:
change(a)
a

[1]

In [2]:
import os.path as osp
from database.transform_factory import get_transform
from database.dataset_factory import get_dataset
from database.loader_factory import get_loader
cfg.TRAIN_TRANSFORM = "resize hflip random_crop to_tensor random_erase normalize"
# cfg.TRAIN_TRANSFORM = "to_tensor random_erase normalize"
# trans = get_transform(cfg, cfg.TRAIN_TRANSFORM)
# dataset = get_dataset(cfg.DB.DATASET)(root=osp.join(cfg.DB.PATH, cfg.DB.DATA), train=True, transform=trans, download=True)
loader = get_loader(cfg.DB.LOADER)(cfg)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
batch = next(iter(loader['train']))

In [7]:
batch['target'].shape

torch.Size([256])

In [6]:
dataset[0][0].shape

torch.Size([3, 32, 32])

In [3]:
trans

Compose(
    Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.2435, 0.2616])
    ToTensor()
    RandomCrop(size=(32, 32), padding=4)
    RandomHorizontalFlip(p=0.5)
    Resize(size=(32, 32), interpolation=PIL.Image.BILINEAR)
)

In [5]:
import sys
sys.path.insert(0, '/media/allen/mass/deep-learning-works/')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist, pdist
import importlib
from config.config_manager import _C as cfg
from tools.utils import deploy_macro
from tools.logger import setup_logger
logger = setup_logger('.')
cfg.merge_from_file("../keypoint.yml")
cfg.RESUME = ""
cfg.MODEL.NORM = 'BN'
cfg.MODEL.GPU = [0,1]
deploy_macro(cfg)

2019-12-04 10:59:19,958 logger INFO: Using GPU: 0,1


In [6]:
# from data.build_loader import build_coco_person_loader
from model.managers.manager_center import CenterManager

In [7]:
# train_loader, val_loader = build_coco_person_loader(cfg)
manager = CenterManager(cfg)

2019-12-04 10:59:26,725 logger INFO: Evaluating model from /home/allen/CenterNet_CornerNet-Squeeze_COCO.pth
2019-12-04 10:59:27,491 logger INFO: pre.0.conv.weight                                       ...... loaded
2019-12-04 10:59:27,493 logger INFO: pre.0.bn.weight                                         ...... skipped
2019-12-04 10:59:27,495 logger INFO: pre.0.bn.bias                                           ...... skipped
2019-12-04 10:59:27,496 logger INFO: pre.0.bn.running_mean                                   ...... skipped
2019-12-04 10:59:27,497 logger INFO: pre.0.bn.running_var                                    ...... skipped
2019-12-04 10:59:27,500 logger INFO: pre.1.conv1.weight                                      ...... loaded
2019-12-04 10:59:27,501 logger INFO: pre.1.bn1.weight                                        ...... skipped
2019-12-04 10:59:27,502 logger INFO: pre.1.bn1.bias                                          ...... skipped
2019-12-04 10:59:27,503 logger

2019-12-04 10:59:27,596 logger INFO: hg_mods.0.low1.1.bn1.weight                             ...... loaded
2019-12-04 10:59:27,597 logger INFO: hg_mods.0.low1.1.bn1.bias                               ...... loaded
2019-12-04 10:59:27,599 logger INFO: hg_mods.0.low1.1.bn1.running_mean                       ...... loaded
2019-12-04 10:59:27,600 logger INFO: hg_mods.0.low1.1.bn1.running_var                        ...... loaded
2019-12-04 10:59:27,602 logger INFO: hg_mods.0.low1.1.conv_1x1.weight                        ...... loaded
2019-12-04 10:59:27,617 logger INFO: hg_mods.0.low1.1.conv_3x3.weight                        ...... loaded
2019-12-04 10:59:27,618 logger INFO: hg_mods.0.low1.1.bn2.weight                             ...... loaded
2019-12-04 10:59:27,619 logger INFO: hg_mods.0.low1.1.bn2.bias                               ...... loaded
2019-12-04 10:59:27,620 logger INFO: hg_mods.0.low1.1.bn2.running_mean                       ...... loaded
2019-12-04 10:59:27,621 logger INFO: 

2019-12-04 10:59:27,714 logger INFO: hg_mods.0.low2.low2.up1.1.conv_1x1.weight               ...... loaded
2019-12-04 10:59:27,715 logger INFO: hg_mods.0.low2.low2.up1.1.conv_3x3.weight               ...... loaded
2019-12-04 10:59:27,716 logger INFO: hg_mods.0.low2.low2.up1.1.bn2.weight                    ...... loaded
2019-12-04 10:59:27,717 logger INFO: hg_mods.0.low2.low2.up1.1.bn2.bias                      ...... loaded
2019-12-04 10:59:27,717 logger INFO: hg_mods.0.low2.low2.up1.1.bn2.running_mean              ...... loaded
2019-12-04 10:59:27,718 logger INFO: hg_mods.0.low2.low2.up1.1.bn2.running_var               ...... loaded
2019-12-04 10:59:27,719 logger INFO: hg_mods.0.low2.low2.low1.0.conv1.weight                 ...... loaded
2019-12-04 10:59:27,720 logger INFO: hg_mods.0.low2.low2.low1.0.bn1.weight                   ...... loaded
2019-12-04 10:59:27,721 logger INFO: hg_mods.0.low2.low2.low1.0.bn1.bias                     ...... loaded
2019-12-04 10:59:27,728 logger INFO: 

2019-12-04 10:59:27,813 logger INFO: hg_mods.0.low2.low2.low2.low1.1.bn2.running_mean        ...... loaded
2019-12-04 10:59:27,814 logger INFO: hg_mods.0.low2.low2.low2.low1.1.bn2.running_var         ...... loaded
2019-12-04 10:59:27,816 logger INFO: hg_mods.0.low2.low2.low2.low2.0.conv1.weight            ...... loaded
2019-12-04 10:59:27,817 logger INFO: hg_mods.0.low2.low2.low2.low2.0.bn1.weight              ...... loaded
2019-12-04 10:59:27,818 logger INFO: hg_mods.0.low2.low2.low2.low2.0.bn1.bias                ...... loaded
2019-12-04 10:59:27,819 logger INFO: hg_mods.0.low2.low2.low2.low2.0.bn1.running_mean        ...... loaded
2019-12-04 10:59:27,819 logger INFO: hg_mods.0.low2.low2.low2.low2.0.bn1.running_var         ...... loaded
2019-12-04 10:59:27,821 logger INFO: hg_mods.0.low2.low2.low2.low2.0.conv_1x1.weight         ...... loaded
2019-12-04 10:59:27,823 logger INFO: hg_mods.0.low2.low2.low2.low2.0.conv_3x3.weight         ...... loaded
2019-12-04 10:59:27,824 logger INFO: 

2019-12-04 10:59:27,924 logger INFO: hg_mods.0.low2.low2.low3.0.conv1.weight                 ...... loaded
2019-12-04 10:59:27,926 logger INFO: hg_mods.0.low2.low2.low3.0.bn1.weight                   ...... loaded
2019-12-04 10:59:27,927 logger INFO: hg_mods.0.low2.low2.low3.0.bn1.bias                     ...... loaded
2019-12-04 10:59:27,928 logger INFO: hg_mods.0.low2.low2.low3.0.bn1.running_mean             ...... loaded
2019-12-04 10:59:27,929 logger INFO: hg_mods.0.low2.low2.low3.0.bn1.running_var              ...... loaded
2019-12-04 10:59:27,931 logger INFO: hg_mods.0.low2.low2.low3.0.conv_1x1.weight              ...... loaded
2019-12-04 10:59:27,932 logger INFO: hg_mods.0.low2.low2.low3.0.conv_3x3.weight              ...... loaded
2019-12-04 10:59:27,934 logger INFO: hg_mods.0.low2.low2.low3.0.bn2.weight                   ...... loaded
2019-12-04 10:59:27,936 logger INFO: hg_mods.0.low2.low2.low3.0.bn2.bias                     ...... loaded
2019-12-04 10:59:27,938 logger INFO: 

...

2019-12-04 10:59:28,422 logger INFO: hg_mods.1.low2.low3.0.bn2.running_var                   ...... loaded
2019-12-04 10:59:28,422 logger INFO: hg_mods.1.low2.low3.1.conv1.weight                      ...... loaded
2019-12-04 10:59:28,424 logger INFO: hg_mods.1.low2.low3.1.bn1.weight                        ...... loaded
2019-12-04 10:59:28,425 logger INFO: hg_mods.1.low2.low3.1.bn1.bias                          ...... loaded
2019-12-04 10:59:28,426 logger INFO: hg_mods.1.low2.low3.1.bn1.running_mean                  ...... loaded
2019-12-04 10:59:28,427 logger INFO: hg_mods.1.low2.low3.1.bn1.running_var                   ...... loaded
2019-12-04 10:59:28,427 logger INFO: hg_mods.1.low2.low3.1.conv_1x1.weight                   ...... loaded
2019-12-04 10:59:28,428 logger INFO: hg_mods.1.low2.low3.1.conv_3x3.weight                   ...... loaded
2019-12-04 10:59:28,429 logger INFO: hg_mods.1.low2.low3.1.bn2.weight                        ...... skipped
2019-12-04 10:59:28,430 logger INFO:

2019-12-04 10:59:28,529 logger INFO: pre.0.bn.0.running_mean                                 ...... not loaded
2019-12-04 10:59:28,530 logger INFO: pre.0.bn.0.running_var                                  ...... not loaded
2019-12-04 10:59:28,530 logger INFO: pre.0.bn.0.num_batches_tracked                          ...... not loaded
2019-12-04 10:59:28,532 logger INFO: pre.1.bn1.0.weight                                      ...... not loaded
2019-12-04 10:59:28,534 logger INFO: pre.1.bn1.0.bias                                        ...... not loaded
2019-12-04 10:59:28,535 logger INFO: pre.1.bn1.0.running_mean                                ...... not loaded
2019-12-04 10:59:28,535 logger INFO: pre.1.bn1.0.running_var                                 ...... not loaded
2019-12-04 10:59:28,536 logger INFO: pre.1.bn1.0.num_batches_tracked                         ...... not loaded
2019-12-04 10:59:28,537 logger INFO: pre.1.bn2.num_batches_tracked                           ...... not loaded
2

2019-12-04 10:59:28,592 logger INFO: hg_mods.0.low2.low2.low2.low2.1.bn2.num_batches_tracked ...... not loaded
2019-12-04 10:59:28,593 logger INFO: hg_mods.0.low2.low2.low2.low2.2.bn1.num_batches_tracked ...... not loaded
2019-12-04 10:59:28,593 logger INFO: hg_mods.0.low2.low2.low2.low2.2.bn2.num_batches_tracked ...... not loaded
2019-12-04 10:59:28,594 logger INFO: hg_mods.0.low2.low2.low2.low2.3.bn1.num_batches_tracked ...... not loaded
2019-12-04 10:59:28,594 logger INFO: hg_mods.0.low2.low2.low2.low2.3.bn2.num_batches_tracked ...... not loaded
2019-12-04 10:59:28,595 logger INFO: hg_mods.0.low2.low2.low2.low3.0.bn1.num_batches_tracked ...... not loaded
2019-12-04 10:59:28,595 logger INFO: hg_mods.0.low2.low2.low2.low3.0.bn2.num_batches_tracked ...... not loaded
2019-12-04 10:59:28,596 logger INFO: hg_mods.0.low2.low2.low2.low3.1.bn1.num_batches_tracked ...... not loaded
2019-12-04 10:59:28,598 logger INFO: hg_mods.0.low2.low2.low2.low3.1.bn2.0.weight            ...... not loaded
2

2019-12-04 10:59:28,665 logger INFO: hg_mods.1.low2.low2.low2.up1.1.bn2.num_batches_tracked  ...... not loaded
2019-12-04 10:59:28,665 logger INFO: hg_mods.1.low2.low2.low2.low1.0.bn1.num_batches_tracked ...... not loaded
2019-12-04 10:59:28,666 logger INFO: hg_mods.1.low2.low2.low2.low1.0.bn2.0.weight            ...... not loaded
2019-12-04 10:59:28,667 logger INFO: hg_mods.1.low2.low2.low2.low1.0.bn2.0.bias              ...... not loaded
2019-12-04 10:59:28,667 logger INFO: hg_mods.1.low2.low2.low2.low1.0.bn2.0.running_mean      ...... not loaded
2019-12-04 10:59:28,668 logger INFO: hg_mods.1.low2.low2.low2.low1.0.bn2.0.running_var       ...... not loaded
2019-12-04 10:59:28,668 logger INFO: hg_mods.1.low2.low2.low2.low1.0.bn2.0.num_batches_tracked ...... not loaded
2019-12-04 10:59:28,669 logger INFO: hg_mods.1.low2.low2.low2.low1.1.bn1.num_batches_tracked ...... not loaded
2019-12-04 10:59:28,670 logger INFO: hg_mods.1.low2.low2.low2.low1.1.bn2.num_batches_tracked ...... not loaded

2019-12-04 10:59:28,723 logger INFO: wh.0.0.conv.bias                                        ...... not loaded
2019-12-04 10:59:28,724 logger INFO: wh.0.1.weight                                           ...... not loaded
2019-12-04 10:59:28,727 logger INFO: wh.0.1.bias                                             ...... not loaded
2019-12-04 10:59:28,727 logger INFO: wh.1.0.conv.weight                                      ...... not loaded
2019-12-04 10:59:28,727 logger INFO: wh.1.0.conv.bias                                        ...... not loaded
2019-12-04 10:59:28,728 logger INFO: wh.1.1.weight                                           ...... not loaded
2019-12-04 10:59:28,729 logger INFO: wh.1.1.bias                                             ...... not loaded
2019-12-04 10:59:28,729 logger INFO: reg.0.0.conv.weight                                     ...... not loaded
2019-12-04 10:59:28,730 logger INFO: reg.0.0.conv.bias                                       ...... not loaded
2

In [10]:
manager.use_multigpu()
model = manager.model

2019-12-04 10:59:58,299 logger INFO: Using GPU: 0,1
2019-12-04 10:59:58,302 logger INFO: Use Multi-GPUs


In [11]:
model

DataParallel(
  (module): Model(
    (backbone): HourglassNet(
      (pre): Sequential(
        (0): convolution(
          (conv): Conv2d(3, 128, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn): Sequential(
            (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): ReLU(inplace)
          )
        )
        (1): residual(
          (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn1): Sequential(
            (0): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): ReLU(inplace)
          )
          (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (skip): Sequential(
            (0): Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
           

In [6]:
batch = next(iter(train_loader))

In [7]:
for key in batch.keys():
    batch[key] = batch[key].cuda()

In [12]:
images = batch['inp'].cuda()
feats = manager.model(images) 


In [9]:
pre_out = model.backbone.pre(images)
hg1_up1_out = model.backbone.hg_mods[0].up1(pre_out)
hg1_low1_out = model.backbone.hg_mods[0].low1(hg1_up1_out)

In [10]:
hg1_low1_out.shape

torch.Size([1, 256, 32, 32])

In [11]:
hg1_low1_out.sum().sigmoid().backward()

In [8]:
batch.keys()

dict_keys(['inp', 'hm', 'wh', 'reg', 'reg_mask', 'ind'])

In [13]:
a, b = manager.loss_func(feats, batch)
# self.total_loss.backward()

In [29]:
feats[0]['reg'].shape

torch.Size([1, 2, 64, 64])

In [14]:
a.backward()

In [15]:
num_params = 0
for layer, p in model.named_parameters():

    if not p.requires_grad:
        continue
    
    num_params += p.numel()

In [16]:
print("Trainable parameters: {:.2f}M".format(num_params / 1000000.0))

Trainable parameters: 25.13M


In [17]:
a = torch.rand(1,3,512,512)
model = manager.model
model.train()

Model(
  (backbone): HourglassNet(
    (pre): Sequential(
      (0): convolution(
        (conv): Conv2d(3, 128, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn): InPlaceABN(128, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
      )
      (1): residual(
        (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): InPlaceABN(256, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
        (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): InPlaceABN(256, eps=1e-05, momentum=0.1, affine=True, activation=identity)
        (skip): Sequential(
          (0): Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): InPlaceABN(256, eps=1e-05, momentum=0.1, affine=True, activation=identity)
        )
        (relu): LeakyReLU(negative_slope=0.01, inplace)
      )
      (2): residual(
        (conv1): Conv2d(

In [18]:
model.heads

AttributeError: 'Model' object has no attribute 'heads'

In [19]:
out = model(a.cuda())

[{'hm': tensor([[[[-2.2494, -2.2093, -2.2957,  ..., -2.1749, -2.2129, -2.1252],
            [-2.1748, -2.1643, -2.2896,  ..., -2.1925, -2.2053, -2.1763],
            [-2.2744, -2.3161, -2.1912,  ..., -2.2263, -2.1701, -2.1511],
            ...,
            [-2.0581, -2.0498, -2.1303,  ..., -2.2343, -2.1853, -2.1726],
            [-2.1471, -2.0332, -2.0911,  ..., -2.1924, -2.2166, -2.1311],
            [-2.1272, -2.1599, -2.0723,  ..., -2.1747, -2.1571, -2.1465]]]],
         device='cuda:0', grad_fn=<CudnnConvolutionBackward>),
  'wh': tensor([[[[-0.1594, -0.0312, -0.2108,  ...,  0.0093, -0.0513, -0.0817],
            [-0.1555, -0.0910, -0.1681,  ..., -0.0044,  0.0057, -0.0378],
            [-0.1741, -0.1662, -0.1543,  ..., -0.0533,  0.0912, -0.0051],
            ...,
            [ 0.0124,  0.0208,  0.0105,  ..., -0.0302,  0.0594,  0.0097],
            [ 0.0253,  0.0664,  0.0331,  ..., -0.0059,  0.0351,  0.0471],
            [ 0.0057,  0.0295, -0.0183,  ..., -0.0148,  0.0399, -0.0286]],

In [12]:
out[-1]

{'hm': tensor([[[[  115.4680,  -392.3594,   334.7852,  ...,  -376.1155,
               35.9382,  -213.9702],
           [ -248.1862,  -124.4669,  -217.4783,  ...,   216.5553,
             -145.1919,   231.8074],
           [   15.1993,  -173.6681,   -26.8982,  ..., -1134.5842,
              167.5008,  -186.2161],
           ...,
           [  196.9365,  -217.4518,   243.7393,  ...,  -400.8627,
               62.2878,   515.2735],
           [  652.4150,  -492.6696,  -289.9304,  ..., -2323.4958,
             -366.9826,  -329.8666],
           [   33.6572,   138.9205,  -601.0677,  ...,    71.5848,
             -119.8921,   451.1734]]]], device='cuda:0'),
 'wh': tensor([[[[  172.3305,  -178.0389,   257.0131,  ...,  -406.8870,
             -123.4730,  -160.6042],
           [ -111.5011,   -12.0172,    60.4420,  ...,  -390.6871,
              150.5281,  -313.6488],
           [  333.1461,   514.1946,   277.1033,  ...,   184.2935,
             -340.0359,   -61.7338],
           ...,
        

In [8]:
cache_file = '/home/allen/CornerNet_Squeeze_500000.pkl'
print("loading model from {}".format(cache_file))
with open(cache_file, "rb") as f:
    state = torch.load(f)
    

loading model from /home/allen/CornerNet_Squeeze_500000.pkl


In [18]:
src_state = state
dst_state = model.module.backbone.state_dict()

In [19]:
dst_layers = []
for key in dst_state.keys():
    if 'pre' in key or 'hg_mods' in key or 'cnvs' in key or 'inters' in key:
        if 'num_batches_tracked' not in key:
            dst_layers.append(key)

In [21]:
src_layers = []
for key in src_state.keys():
    if 'pre' in key or 'hgs' in key or 'cnvs' in key or 'inters' in key:
        if 'num_batches_tracked' not in key:
            src_layers.append(key)

In [20]:
dst_layers

['pre.0.conv.weight',
 'pre.0.bn.0.weight',
 'pre.0.bn.0.bias',
 'pre.0.bn.0.running_mean',
 'pre.0.bn.0.running_var',
 'pre.1.conv1.weight',
 'pre.1.bn1.0.weight',
 'pre.1.bn1.0.bias',
 'pre.1.bn1.0.running_mean',
 'pre.1.bn1.0.running_var',
 'pre.1.conv2.weight',
 'pre.1.bn2.weight',
 'pre.1.bn2.bias',
 'pre.1.bn2.running_mean',
 'pre.1.bn2.running_var',
 'pre.1.skip.0.weight',
 'pre.1.skip.1.weight',
 'pre.1.skip.1.bias',
 'pre.1.skip.1.running_mean',
 'pre.1.skip.1.running_var',
 'pre.2.conv1.weight',
 'pre.2.bn1.0.weight',
 'pre.2.bn1.0.bias',
 'pre.2.bn1.0.running_mean',
 'pre.2.bn1.0.running_var',
 'pre.2.conv2.weight',
 'pre.2.bn2.weight',
 'pre.2.bn2.bias',
 'pre.2.bn2.running_mean',
 'pre.2.bn2.running_var',
 'pre.2.skip.0.weight',
 'pre.2.skip.1.weight',
 'pre.2.skip.1.bias',
 'pre.2.skip.1.running_mean',
 'pre.2.skip.1.running_var',
 'hg_mods.0.up1.0.conv1.weight',
 'hg_mods.0.up1.0.bn1.weight',
 'hg_mods.0.up1.0.bn1.bias',
 'hg_mods.0.up1.0.bn1.running_mean',
 'hg_mods.0.u

In [22]:
src_layers

['module.hg.pre.0.conv.weight',
 'module.hg.pre.0.bn.weight',
 'module.hg.pre.0.bn.bias',
 'module.hg.pre.0.bn.running_mean',
 'module.hg.pre.0.bn.running_var',
 'module.hg.pre.1.conv1.weight',
 'module.hg.pre.1.bn1.weight',
 'module.hg.pre.1.bn1.bias',
 'module.hg.pre.1.bn1.running_mean',
 'module.hg.pre.1.bn1.running_var',
 'module.hg.pre.1.conv2.weight',
 'module.hg.pre.1.bn2.weight',
 'module.hg.pre.1.bn2.bias',
 'module.hg.pre.1.bn2.running_mean',
 'module.hg.pre.1.bn2.running_var',
 'module.hg.pre.1.skip.0.weight',
 'module.hg.pre.1.skip.1.weight',
 'module.hg.pre.1.skip.1.bias',
 'module.hg.pre.1.skip.1.running_mean',
 'module.hg.pre.1.skip.1.running_var',
 'module.hg.pre.2.conv1.weight',
 'module.hg.pre.2.bn1.weight',
 'module.hg.pre.2.bn1.bias',
 'module.hg.pre.2.bn1.running_mean',
 'module.hg.pre.2.bn1.running_var',
 'module.hg.pre.2.conv2.weight',
 'module.hg.pre.2.bn2.weight',
 'module.hg.pre.2.bn2.bias',
 'module.hg.pre.2.bn2.running_mean',
 'module.hg.pre.2.bn2.running_va

In [23]:
valid_state = {}
for dst, src in zip(dst_layers, src_layers):
    if dst_state[dst].size() == src_state[src].size():
        valid_state[dst] = src_state[src]
    else:
        print(dst, src)

In [21]:
src_state[src]

tensor([1.4887, 0.4198, 0.5426, 0.5092, 0.4947, 0.3634, 0.3661, 0.2819, 0.3378,
        0.5425, 0.5796, 0.5219, 0.3570, 0.2839, 0.3557, 0.4213, 0.6932, 0.3933,
        0.6667, 0.5858, 0.5530, 0.4547, 0.4733, 0.3944, 0.3859, 0.4886, 0.4198,
        0.6582, 0.4538, 0.3808, 0.6788, 0.6328, 0.8631, 0.4983, 0.5490, 0.3519,
        0.5631, 0.4594, 0.5365, 0.5248, 0.4451, 0.4558, 0.4793, 0.6389, 0.7234,
        0.3734, 0.3457, 0.5360, 0.4281, 1.1684, 0.5294, 0.4066, 0.3297, 0.3671,
        0.6113, 0.4425, 0.4684, 0.6815, 0.3859, 0.6053, 0.3822, 0.3913, 1.2453,
        0.4409, 0.3566, 0.4454, 0.4176, 0.4760, 0.4971, 0.5012, 0.4316, 0.3986,
        0.3751, 0.3391, 0.3834, 0.4656, 0.4162, 0.7432, 0.5051, 0.6335, 0.4490,
        0.3647, 0.4829, 0.8224, 0.3247, 0.3659, 0.4589, 0.3837, 0.4440, 0.4366,
        0.5135, 0.5253, 0.4228, 0.3761, 0.4436, 0.4647, 0.5121, 0.4203, 0.4344,
        0.3311, 0.6444, 0.4212, 0.5446, 0.2708, 0.4052, 0.3880, 0.3720, 0.7506,
        0.4735, 0.4232, 0.3548, 0.5058, 

In [4]:
for key in state.keys():
    print(key)

pre.0.conv.weight
pre.0.bn.weight
pre.0.bn.bias
pre.0.bn.running_mean
pre.0.bn.running_var
pre.1.conv1.weight
pre.1.bn1.weight
pre.1.bn1.bias
pre.1.bn1.running_mean
pre.1.bn1.running_var
pre.1.conv2.weight
pre.1.bn2.weight
pre.1.bn2.bias
pre.1.bn2.running_mean
pre.1.bn2.running_var
pre.1.skip.0.weight
pre.1.skip.1.weight
pre.1.skip.1.bias
pre.1.skip.1.running_mean
pre.1.skip.1.running_var
pre.2.conv1.weight
pre.2.bn1.weight
pre.2.bn1.bias
pre.2.bn1.running_mean
pre.2.bn1.running_var
pre.2.conv2.weight
pre.2.bn2.weight
pre.2.bn2.bias
pre.2.bn2.running_mean
pre.2.bn2.running_var
pre.2.skip.0.weight
pre.2.skip.1.weight
pre.2.skip.1.bias
pre.2.skip.1.running_mean
pre.2.skip.1.running_var
hg_mods.0.up1.0.conv1.weight
hg_mods.0.up1.0.bn1.weight
hg_mods.0.up1.0.bn1.bias
hg_mods.0.up1.0.bn1.running_mean
hg_mods.0.up1.0.bn1.running_var
hg_mods.0.up1.0.conv_1x1.weight
hg_mods.0.up1.0.conv_3x3.weight
hg_mods.0.up1.0.bn2.weight
hg_mods.0.up1.0.bn2.bias
hg_mods.0.up1.0.bn2.running_mean
hg_mods.0.up

In [17]:
for dst, src in zip(dst_layers, src_layers):
    if (dst_state[dst] != src_state[src]).sum() > 0:
        print(dst, src)

In [24]:
torch.save(valid_state, '/home/allen/CenterNet_CornerNet-Squeeze_BN_COCO.pth')

In [3]:
state = torch.load('/home/allen/CenterNet_CornerNet-Squeeze_COCO.pth')